In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive
from transformers import BertTokenizer, TFBertModel, BertModel
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve,f1_score
from sklearn import metrics
from sklearn.utils.class_weight import compute_class_weight
import os
import bert
# from keras_bert import *
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional,GlobalMaxPooling1D
from keras.preprocessing.sequence import pad_sequences
from transformers import XLNetTokenizer, XLNetModel,TFXLNetModel,get_linear_schedule_with_warmup
from transformers import AutoModel, BertTokenizerFast, AdamW
from sklearn.model_selection import train_test_split

from pytorch_transformers import XLNetConfig
# from pytorch_transformers import XLNetTokenizer
from pytorch_transformers import XLNetForSequenceClassification
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import os
import tf_slim as slim
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler, Dataset
import torch.nn as nn
import numpy as np
# drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# !pip3 install tf-models-official
!pip3 install tf_slim
!pip3 install transformers
!pip3 install bert
!pip3 install pytorch_transformers

# SARC dataset

In [ ]:
train = pd.read_csv("train-balanced.csv",sep='\t')
train.columns=['label','comment','author','subreddit','score','ups','downs','date','created_utc','parent_comment']
# t.columns = ['parent','response','label0','label1']
# t['parent'].str.split('|', expand=True)
train[:10]

In [ ]:
test = pd.read_csv("test-balanced.csv",sep='\t')
test.columns=['label','comment','author','subreddit','score','ups','downs','date','created_utc','parent_comment']
test[:10]

In [ ]:
# clean data
train = train.dropna()
train = train.reset_index()
test = test.dropna()
test = test.reset_index()
comments = [train['parent_comment'][i] + '[SEP]' + train['comment'][i] for i in range(len(train))]
train['inputs'] = comments

In [ ]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in comments]

pd.Series(seq_len).hist(bins = 60)

In [ ]:
# obtained from graph of last cell
max_seq_len = 64

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(train['inputs'], train['label'], 
                                                                    random_state=2022, 
                                                                    test_size=0.3, 
                                                                    stratify=train['label'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2022, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

# Lee etc's Model
Ref: 
*   BERT: https://github.com/prateekjoshi565/Fine-Tuning-BERT/blob/master/Fine_Tuning_BERT_for_Spam_Classification.ipynb
*   NeXtVLAD: https://github.com/zhangyaoyuan/NextVLAD-Attention-Model/blob/master/NeXtVLADModelLF.py



In [ ]:
# from transformers.utils.dummy_pt_objects import BertModel
# bert = BertModel.from_pretrained('bert-large-cased')
bert = AutoModel.from_pretrained('bert-large-cased')
# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-large-cased')

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
#define a batch size
batch_size = 4

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True

In [ ]:
## OWN
lr = 1e-6
batch_size = 4
dropout = 0.25
max_document_length = 50  # each sentence has until 100 words
dev_size = 0.8 # split percentage to train\validation data
# max_size = 5000 # maximum vocabulary size
seed = 27
num_classes = 2
lstm_units = 1024
lstm_layers = 2  # LSTM layers
num_epochs = 4
lstm_out = 128
cluster_size = 512  # K
expansion = 4    # lambda
groups = 8       # G
output_size = 2
feature_size = 128  # N
max_frames = 4 

In [ ]:
## Partial codes from original model but reconstructured based on Lee et al.
class Bert_BiLSTM_NeXtVLAD(nn.Module):

    def __init__(self, bert):
      
      super(Bert_BiLSTM_NeXtVLAD, self).__init__()

      self.bert = bert

      # LSTM layers
      self.bilstm = nn.LSTM(128,
                            lstm_units,
                            num_layers=lstm_layers,
                            bidirectional=True,
                            batch_first=True)
      
      # nextvlad params
      self.cluster_size = cluster_size
      self.expansion = expansion
      self.groups = groups
      self.max_frames = max_frames  # M
      self.feature_size = feature_size
      self.is_training = True

      # dropout layer
      self.dropout = nn.Dropout(dropout)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=2)

    def init_hidden(self, batch_size):
        hidden = (Variable(torch.zeros(lstm_layers * 2, batch_size, lstm_units)),
                Variable(torch.zeros(lstm_layers * 2, batch_size, lstm_units)))
        return hidden

    #define the forward pass
    def forward(self, sent_id, mask):
      batch_size = sent_id.size(0)

      # pass the inputs to the bert  
      # x = self.bert(sent_id)
      _, x = self.bert(sent_id, attention_mask=mask, return_dict = False)

      # construct bilstm
      hidden = self.init_hidden(batch_size)

      lstm_out, (hidden_last,cn_last) = self.bilstm(x, hidden)

      hidden_last_L = hidden_last[-2]
      
      hidden_last_R = hidden_last[-1]
      
      hidden_last_out = torch.cat([hidden_last_L,hidden_last_R],dim=-1)

      x = self.dropout(hidden_last_out)[:max_frames]

      # construct nextvlad
      x = slim.fully_connected(x, self.expansion * self.feature_size, activation_fn=None,
                                    weights_initializer=slim.variance_scaling_initializer())

      attention = slim.fully_connected(x, self.groups, activation_fn=tf.nn.sigmoid,
                                         weights_initializer=slim.variance_scaling_initializer())
      
      attention = tf.reshape(attention, [-1, self.max_frames*self.groups, 1])

      tf.summary.histogram("sigmoid_attention", attention)
      
      feature_size = self.expansion * self.feature_size // self.groups

      cluster_weights = tf.get_variable("cluster_weights",
                                        [self.expansion*self.feature_size, self.groups*self.cluster_size],
                                        initializer=slim.variance_scaling_initializer()
                                        )

      # tf.summary.histogram("cluster_weights", cluster_weights)
      reshaped_input = tf.reshape(input, [-1, self.expansion * self.feature_size])
      activation = tf.matmul(reshaped_input, cluster_weights)

      activation = slim.batch_norm(
          activation,
          center=True,
          scale=True,
          is_training=self.is_training,
          scope="cluster_bn",
          fused=False)

      activation = tf.reshape(activation, [-1, self.max_frames * self.groups, self.cluster_size])
      activation = tf.nn.softmax(activation, axis=-1)
      activation = tf.multiply(activation, attention)
      # tf.summary.histogram("cluster_output", activation)
      a_sum = tf.reduce_sum(activation, -2, keep_dims=True)

      cluster_weights2 = tf.get_variable("cluster_weights2",
                                          [1, feature_size, self.cluster_size],
                                          initializer=slim.variance_scaling_initializer()
                                          )
      a = tf.multiply(a_sum, cluster_weights2)

      activation = tf.transpose(activation, perm=[0, 2, 1])

      reshaped_input = tf.reshape(input, [-1, self.max_frames * self.groups, feature_size])
      vlad = tf.matmul(activation, reshaped_input)
      vlad = tf.transpose(vlad, perm=[0, 2, 1])
      vlad = tf.subtract(vlad, a)

      vlad = tf.nn.l2_normalize(vlad, 1)

      vlad = tf.reshape(vlad, [-1, self.cluster_size * feature_size])
      vlad = slim.batch_norm(vlad,
              center=True,
              scale=True,
              is_training=self.is_training,
              scope="vlad_bn",
              fused=False)


      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = Bert_BiLSTM_NeXtVLAD(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = lr)

In [ ]:
#compute the class weights
class_wts = compute_class_weight(class_weight ='balanced', classes=np.unique(train_labels), y=train_labels)

print(class_wts)

In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.CrossEntropyLoss(weight= weights)

# number of training epochs
epochs = 4

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
# #load weights of best model
# path = 'saved_weights.pt'
# model.load_state_dict(torch.load(path))

# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()


# F1 evaluate
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

# DataData Scrapping 

In [ ]:
# 0 - Neutral, 1 - Postive, -1 - Negative
sent_data = pd.read_csv("Reddit_Data.csv")
sent_data=sent_data.dropna()
sent_data[:10]

In [ ]:
# sentences  = []
sent_data = sent_data[sent_data['category'] !=0]
# for sentence in sent_data['clean_comment']:
#   sentence = sentence + "[SEP] [CLS]"
#   sentences.append(sentence)
labels = sent_data['category'].values
sentences = list(sent_data['clean_comment'])

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
MAX_LEN = 128 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2022, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2022, test_size=0.1)
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



# Sentiment Analysis Model - XLNet

* XLNet: https://www.kaggle.com/code/rihab147/xlnet-tunisian-sentiment-analysis/notebook & https://github.com/zihangdai/xlnet



In [ ]:
# only needs to be done once
# ! wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
# ! unzip cased_L-24_H-1024_A-16.zip 
# ! git clone https://github.com/zihangdai/xlnet.git

In [ ]:
SCRIPTS_DIR = 'xlnet' #@param {type:"string"}
DATA_DIR = 'DATAPATH' #@param {type:"string"}
OUTPUT_DIR = 'proc_data/reddit' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'xlnet_cased_L-24_H-1024_A-16' #@param {type:"string"}
CHECKPOINT_DIR = 'exp/reddit' #@param {type:"string"}

In [ ]:
# generate predicted label
train_command = "python xlnet/run_classifier.py \
  --do_train=True \
  --do_eval=True \
  --eval_all_ckpt=True \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=8 \
  --eval_batch_size=8 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=4000 \
  --warmup_steps=500 \
  --save_steps=500 \
  --iterations=500"

! {train_command}

In [ ]:
# 0 - Neutral, 1 - Postive, -1 - Negative
sent_data = pd.read_csv("Reddit_Data.csv")
sent_data=sent_data.dropna()

In [ ]:
# sentences  = []
sent_data = sent_data[sent_data['category'] !=0]
# for sentence in sent_data['clean_comment']:
#   sentence = sentence + "[SEP] [CLS]"
#   sentences.append(sentence)
labels = sent_data['category'].values
sentences = list(sent_data['clean_comment'])

In [ ]:
MAX_LEN = 128 

In [ ]:
## Method 1

def get_inputs(comments, tokenizer, max_len=MAX_LEN):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len,pad_to_max_length=True, add_special_tokens=True) for t in comments]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

def warmup(epoch, lr):
    """Used for increasing the learning rate slowly, this tends to achieve better convergence.
    However, as we are finetuning for few epoch it's not crucial.
    """
    return max(lr +1e-6, 2e-5)

def plot_metrics(pred, true_labels):
    """Plots a ROC curve with the accuracy and the AUC"""
    acc = accuracy_score(true_labels, np.array(pred.flatten() >= .5, dtype='int'))
    fpr, tpr, thresholds = roc_curve(true_labels, pred)
    auc = roc_auc_score(true_labels, pred)

    fig, ax = plt.subplots(1, figsize=(8,8))
    ax.plot(fpr, tpr, color='red')
    ax.plot([0,1], [0,1], color='black', linestyle='--')
    ax.set_title(f"AUC: {auc}\nACC: {acc}");
    return fig

In [ ]:
def create_xlnet(mname):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_inputs = tf.keras.Input(shape=(MAX_LEN,), name='word_inputs', dtype='int32')

    # Call XLNet model
    xlnet = TFXLNetModel.from_pretrained(mname)
    xlnet_encodings = xlnet(word_inputs)[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    # Apply dropout for regularization
    doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    # Final output 
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

    # Compile model
    model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), ])

    return model

In [ ]:
xlnet_model = 'xlnet-large-cased'
xlnet_tokenizer = XLNetTokenizer.from_pretrained(xlnet_model)
xlnet = create_xlnet(xlnet_model)
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.15, random_state=122)
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

inp_tok, ids, segments = get_inputs(X_train, xlnet_tokenizer)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.02, restore_best_weights=True),
    tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=1e-6, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-6)
]

In [ ]:
hist = xlnet.fit(x=inp_tok, y=y_train, epochs=1, batch_size=16, validation_split=.3, callbacks=callbacks)

In [ ]:
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams
from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.utils import shuffle

In [ ]:
RANDOM_SEED = 73
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

df = shuffle(sent_data)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
MAX_LEN = 512
class ImdbDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True,
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = ImdbDataset(
    reviews=df.clean_comment.to_numpy(),
    targets=df.category.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
BATCH_SIZE = 4

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
from transformers import XLNetForSequenceClassification
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

In [ ]:
EPOCHS = 3

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-6)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [ ]:
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(4,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [ ]:

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train),
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '/content/drive/My Drive/NLP/Sentiment Analysis Series/models/xlnet_model.bin')
        best_accuracy = val_acc

In [ ]:
data = next(iter(val_data_loader))

dict_keys(['review_text', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
targets = data['targets'].to(device)
print(input_ids.reshape(4,512).shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length